<a href="https://colab.research.google.com/github/HansHenseler/masdav2022/blob/main/Part_4_Exercise_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Opensearch and log2timeline

Exercise 4: 

Master of Advanced Studies in Digital Forensics & Cyber Investigation

Data Analytics and Visualization for Digital Forensics

(c) Hans Henseler, 2022


## 1 The examples are in the Part 4 notebook. This notebook only contains initializations needed to run the exercises at the end

First install Plaso-tools as we did in exercise 3

In [ ]:
# various install steps to install plaso tools and dependencies to get plaso working in colab
# -y option is to skip user interaction
# some packages need to be deinstalled and reinstalled to resolve dependencies
# these steps take app. 3 minutes to complete on a fresh colab instance
!add-apt-repository -y ppa:gift/stable
!apt update
!apt-get update
!apt install plaso-tools
!pip uninstall -y pytsk3
!pip install pytsk3
!pip uninstall -y yara-python
!pip install yara-python
!pip uninstall -y lz4
!pip install lz4

In [ ]:
# This notebook was tested with version 20220724 (psort.py assumes opensearch and longer elasticsearch)

!psort.py -V

plaso - psort version 20220724


In [ ]:
# check if plaso tools were installed by running psort.py

!psort.py -o list


******************************** Output Modules ********************************
         Name : Description
--------------------------------------------------------------------------------
      dynamic : Dynamic selection of fields for a separated value output
                format.
         json : Saves the events into a JSON format.
    json_line : Saves the events into a JSON line format.
          kml : Saves events with geography data into a KML format.
       l2tcsv : CSV format used by legacy log2timeline, with 17 fixed fields.
       l2ttln : Extended TLN 7 field | delimited output.
         null : Output module that does not output anything.
   opensearch : Saves the events into an OpenSearch database.
opensearch_ts : Saves the events into an OpenSearch database for use with
                Timesketch.
        rawpy : native (or "raw") Python output.
          tln : TLN 5 field | delimited output.
         xlsx : Excel Spreadsheet (XLSX) output
----------------------------

In [ ]:
!wget -q https://artifacts.opensearch.org/releases/bundle/opensearch/2.2.0/opensearch-2.2.0-linux-x64.tar.gz
!wget -q https://artifacts.opensearch.org/releases/bundle/opensearch/2.2.0/opensearch-2.2.0-linux-x64.tar.gz.sha512
!tar -zxf opensearch-2.2.0-linux-x64.tar.gz
!shasum -a 512 -c opensearch-2.2.0-linux-x64.tar.gz.sha512

opensearch-2.2.0-linux-x64.tar.gz: OK


In [ ]:
# change the owner of the Opensearch filetree from root to daemon because Opensearch cannot run as root.

!sudo chown -R daemon:daemon opensearch-2.2.0/

Run Elasticsearch as a daemon process

In [ ]:
import time

In [ ]:
# start Opensearch from the commandline as user daemon. 

!sudo -H -u daemon sh -c  "opensearch-2.2.0/opensearch-tar-install.sh 1> /content/opensearch-2.2.0/logs/os.log 2> /content/opensearch-2.2.0/logs/os.err &"

In [ ]:
# Sleep for few seconds to let the instance start.
time.sleep(20)

Once the instance has been started, grep for opensearch in the processes list to confirm the availability.

In [ ]:
!ps -ef | grep opensearch

daemon      8014       1 99 18:55 ?        00:00:38 /content/opensearch-2.2.0/jdk/bin/java -Xshare:auto -Dopensearch.networkaddress.cache.ttl=60 -Dopensearch.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/opensearch-7220716448761113169 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecount=32,filesize=64m -Dclk.tck=100 -Djdk.attach.allowAttachSelf=true -Djava.security.policy=/content

query the base endpoint to retrieve information about the cluster.

In [ ]:
!curl -XGET https://localhost:9200 -u 'admin:admin' --insecure

{
  "name" : "74aabc422dc4",
  "cluster_name" : "opensearch",
  "cluster_uuid" : "HRYLb7jXTrqm2LBPZpXDNg",
  "version" : {
    "distribution" : "opensearch",
    "number" : "2.2.0",
    "build_type" : "tar",
    "build_hash" : "b1017fa3b9a1c781d4f34ecee411e0cdf930a515",
    "build_date" : "2022-08-09T02:27:25.256769336Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.10.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "The OpenSearch Project: https://opensearch.org/"
}


In [ ]:
# After installation there is only one index that is the  .opendistro_security index that is used for internal purposes

!curl -XGET "https://localhost:9200/_cat/indices?v" -u 'admin:admin' --insecure

health status index                uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   .opendistro_security 2YfFzTEPRkWRdZQTSQui3A   1   0         10            0     68.3kb         68.3kb


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# In part 3 (step 3) we stored the mus2019ctf.plaso file in your drive. 
#
plaso_file = 'gdrive/MyDrive/mus2019ctf.plaso'
#
# and check if it's there
#
!ls -l $plaso_file

-rw------- 1 root root 403488768 Aug 15 15:17 gdrive/MyDrive/mus2019ctf.plaso


In [ ]:
# If it's not there you can create it by repeating the following steps
# 
# The complete mus2019ctf.plaso file is 450MB and takes a while. After you have created it
# it makes sense to store it in your gdrive so you can reuse it:
#
#plaso_file = 'gdrive/MyDrive/Colab\ Notebooks/Data\ Analytics\ and\ Visualisation\ Course/mus2019ctf.plaso'
#
# if not you need to create it with log2timeline.py using the complete windows_filter.txt filter
#
# add a shortcut in your Google drive to this shared drive https://drive.google.com/drive/folders/1KUlZUl4Sy2JzgbuRW-oHjIGFClY2bl75?usp=sharing
# then mount you google drive in this colab (you need to authorize this colab to access your google drive)
#
#disk_image = "/content/gdrive/MyDrive/Images/Windows/MUS-CTF-19-DESKTOP-001.E01"
#plaso_gdrive_folder = 'gdrive/MyDrive'
#!wget "https://raw.githubusercontent.com/mark-hallman/plaso_filters/master/filter_windows.txt"
#!log2timeline.py -f filter_windows.txt --storage-file mus2019ctf.plaso $disk_image 
#!ls -l $disk_image
#!cp mus2019ctf.plaso $plaso_gdrive_folder
#plaso_file = 'gdrive/MyDrive/mus2019ctf.plaso'
#!ls -l $plaso_file

Use psort to write events to Elasticsearch that we setup earlier. We can use the elastic output format

In [ ]:
# run psort.py. It takes about 11-13 minutes to export all rows from the 385MB plaso file to Opensearch

#
!psort.py -o opensearch --server localhost --port 9200 --opensearch-user admin --opensearch-password admin --opensearch_mappings /usr/share/plaso/opensearch.mappings --use_ssl --ca_certificates_file_path /content/opensearch-2.2.0/config/root-ca.pem --index_name newmus2019ctf $plaso_file --status_view none


2022-08-16 18:58:49,694 [INFO] (MainProcess) PID:8576 <data_location> Determined data location: /usr/share/plaso
2022-08-16 18:58:50,299 [WARNING] (MainProcess) PID:8576 <opensearch_output> Note that specifying your OpenSearch password via --opensearch_password or the environment PLASO_OPENSEARCH_PASSWORD can expose the password to other users on the system.
Processing completed.


In [ ]:
# Let's take a look again at the indices in our Elasticsearch instance
#
#!curl -X GET "https://localhost:9200/_cat/indices?format=json&pretty" -u admin:admin --insecure
!curl -X GET "https://localhost:9200/_cat/indices?v" -u admin:admin --insecure

health status index                        uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   security-auditlog-2022.08.16 VYWbw5tgQ4ykkEY6rM4vBw   1   1         40            0    168.4kb        168.4kb
yellow open   newmus2019ctf                kgfrzDnvRZa0KZVwkfJjKw   1   1     524518            0    366.2mb        366.2mb
green  open   .opendistro_security         2YfFzTEPRkWRdZQTSQui3A   1   0         10            0     69.1kb         69.1kb


In [ ]:
# we can also see what fields were mapped in this index by Psort.py
#
!curl -XGET "https://localhost:9200/newmus2019ctf/_mapping?format=json&pretty" -u admin:admin --insecure

{
  "newmus2019ctf" : {
    "mappings" : {
      "properties" : {
        "access_count" : {
          "type" : "long"
        },
        "account_rid" : {
          "type" : "long"
        },
        "application" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword"
            }
          }
        },
        "binary_path" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "birth_droid_file_identifier" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "birth_droid_volume_identifier" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }


In [ ]:
# So far we have been accessing information directly with curl from the Opensearch REST API
# The is also an Opensearch Python API that we can use. See https://opensearch.org/docs/latest/clients/python/
#

from opensearchpy import OpenSearch

host = 'localhost'
port = 9200
auth = ('admin', 'admin') # For testing only. Don't store credentials in code.
ca_certs_path = '/content/opensearch-2.2.0/config/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': 'localhost', 'port': 9200}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    ca_certs = ca_certs_path
)

In [ ]:
# We define a Python function to list results
#
def print_results(response):
  for num, doc in enumerate(response['hits']['hits']):
    print(num, '-->', doc['_source']) 

def print_results_detailed(response):
  for num, doc in enumerate(response['hits']['hits']):
    print('\n---------------------------------------------------------------------------------------------------\nresult numer: ',num) 
    for key, val in doc['_source'].items():
      print(key, val)

def print_facets(agg_dict):
  sum=0
  for field, val in agg_dict:
      print("facets of field ", field,':')
      for bucket in val['buckets']:
        for key in bucket:
          if key=='key':
            print('\t',bucket[key],end='=')
          else:
            print(bucket[key],end='')
            sum = sum + bucket[key]
        print()
      print("total number of hits for ",field," is ",sum)

def print_hit_stats(response):
  print('hit stats:')
  for key, val in response['hits'].items():
      if key=='hits' :
        print(len(val))
      else:
        print(key, val)
  print('\n')

# Exercises

## 1 Use elasticsearch to filter events in between 2019-03-12 and 2019-03-22

In [ ]:
# see https://opensearch.org/docs/2.0/opensearch/rest-api/search/ and find track_total_hits to get the total number of hits in stead of the max 10.000

query = '{"query": { "query_string": {"query": "datetime:[2019-03-12 TO 2019-03-22]"  }}}'

response = client.search(index="newmus2019ctf", body=query, size=0,track_total_hits=True)
print_hit_stats(response)
print_results_detailed(response)

hit stats:
total {'value': 367946, 'relation': 'eq'}
max_score None
0




## 2 Write a query that performs an aggregation on source_long and source_short (can you find the right field names?)

In [ ]:
query = '{"query": {"match_all": {}}, "aggs": { "source_short": { "terms": { "field": "source_short.keyword"}},  "source_long": { "terms": { "field": "source_long.keyword"}}}}'

response = client.search(index="newmus2019ctf", body=query, size=0)

print_hit_stats(response)
print_facets(response['aggregations'].items())

hit stats:
total {'value': 10000, 'relation': 'gte'}
max_score None
0


facets of field  source_long :
	 NTFS USN change=292405
	 Registry Key=281913
	 WinEVTX=222198
	 File stat=3804
	 MSIE WebCache container record=1081
	 WinPrefetch=839
	 AppCompatCache Registry Entry=742
	 Registry Key - Service=588
	 Task Cache=537
	 System=257
total number of hits for  source_long  is  804364
facets of field  source_short :
	 FILE=296260
	 REG=283847
	 EVT=222198
	 LOG=1302
	 WEBHIST=1172
	 LNK=96
	 PE=86
	 OLECF=41
	 AMCACHE=8
total number of hits for  source_short  is  1609374


## 3 Combine your date range filter from exercise 1 with facet aggregation in exercise 2

In [ ]:
querystring = '{ "query_string": {"query": "datetime:[2019-03-18 TO 2019-03-19]"  }}'
facets = '"aggs": { "source_short": { "terms": { "field": "source_short.keyword"}},  "source_long": { "terms": { "field": "source_long.keyword"}}}'
query = '{"query": %s,%s}' % (querystring,facets)
print_hit_stats(response)
response = client.search(index="newmus2019ctf", body=query, size=0)

print_facets(response['aggregations'].items())

hit stats:
total {'value': 10000, 'relation': 'gte'}
max_score None
0


facets of field  source_long :
	 NTFS USN change=43212
	 WinEVTX=12808
	 File stat=412
	 MSIE WebCache container record=302
	 Registry Key=250
	 WinPrefetch=230
	 Task Cache=58
	 Background Activity Moderator Registry Entry=21
	 MSIE WebCache containers record=17
	 Windows Shortcut=17
total number of hits for  source_long  is  57327
facets of field  source_short :
	 FILE=43633
	 EVT=12808
	 REG=332
	 WEBHIST=319
	 LOG=232
	 LNK=17
	 OLECF=13
total number of hits for  source_short  is  114681


## 4 ***Advanced*** Use opensearch facet aggregation to create a treemap visualisation of a filtered set of events in the index.

####Step 1
The source_short and source_long look interesting for visualisation let's focus on REG, LOG and FILE and run a query

In [ ]:
facets = '"aggs": {   "source_long": { "terms": { "field": "source_long.keyword"}}, "source_short": { "terms": { "field": "source_short.keyword"}}}'
daterange = 'datetime:[2019-03-12 TO 2019-03-22]'
# FILE:
querystring = '{ "query_string": {"query": "%s AND  source_short:(FILE OR LOG OR REG)"  }}' % daterange
query = '{"query": %s,%s}' % (querystring,facets)
response = client.search(index="newmus2019ctf", body=query, size=0,track_total_hits=True)
print_facets(response['aggregations'].items())

facets of field  source_long :
	 NTFS USN change=292405
	 Registry Key=18055
	 File stat=1880
	 WinPrefetch=597
	 Task Cache=158
	 Registry Key - Service=65
	 Background Activity Moderator Registry Entry=36
	 File entry shell item=25
	 Windows Setupapi Log=18
	 System=17
total number of hits for  source_long  is  313256
facets of field  source_short :
	 FILE=294310
	 REG=18335
	 LOG=633
total number of hits for  source_short  is  626534


Why is this not very helpful for a treemap visualisation?

Answer: The problem is that this breakdown is not very useful for a hierarchical visualisation like treemap or sunburst because we would like to know for each value pair (source_long, source_short) how many documents there are

####Step 2
One approach is to aggregate only accross source_long an run 3 separate queries for source_short equal to REG, LOG and FILE respectively and then combine the results into a single dataframe that we can visualise:

In [ ]:

facets = '"aggs": {   "source_long": { "terms": { "field": "source_long.keyword"}}}'
daterange = 'datetime:[2019-03-12 TO 2019-03-22]'
# FILE:
querystring_file = '{ "query_string": {"query": "%s AND  source_short:FILE"  }}' % daterange
query_file = '{"query": %s,%s}' % (querystring_file,facets)
# LOG:
querystring_log = '{ "query_string": {"query": "%s AND  source_short:LOG"  }}' % daterange
query_log = '{"query": %s,%s}' % (querystring_log,facets)
# REG
querystring_reg = '{ "query_string": {"query": "%s AND  source_short:REG"  }}' % daterange
query_reg = '{"query": %s,%s}' % (querystring_reg,facets)


response_reg = client.search(index="newmus2019ctf", body=query_reg, size=0,track_total_hits=True)
response_file = client.search(index="newmus2019ctf", body=query_file, size=0,track_total_hits=True)
response_log = client.search(index="newmus2019ctf", body=query_log, size=0,track_total_hits=True)


print('Facets for REG:')
print_facets(response_reg['aggregations'].items())

print('Facet for FILE:')
print_facets(response_file['aggregations'].items())

print('Facets for LOG:')
print_facets(response_log['aggregations'].items())

Facets for REG:
facets of field  source_long :
	 Registry Key=18055
	 Task Cache=158
	 Registry Key - Service=65
	 Background Activity Moderator Registry Entry=36
	 AppCompatCache Registry Entry=13
	 Registry Key - Winlogon=3
	 Registry Key - USB Entries=2
	 Registry Key - Run Key=1
	 Registry Key - User Account Information=1
	 Registry Key Shutdown Entry=1
total number of hits for  source_long  is  18335
Facet for FILE:
facets of field  source_long :
	 NTFS USN change=292405
	 File stat=1880
	 File entry shell item=25
total number of hits for  source_long  is  294310
Facets for LOG:
facets of field  source_long :
	 WinPrefetch=597
	 Windows Setupapi Log=18
	 System=17
	 System - Network Connection=1
total number of hits for  source_long  is  633


In [ ]:
# create a variation of the print_facets function that outputs values to a list
# containing source_short, source_long and doc_count
# 
def buckets_to_list(agg_dict,source_short,list):
  for field, val in agg_dict:
    for bucket in val['buckets']:
      list.append([source_short,bucket['key'],bucket['doc_count']])
  return list
  
data1 = []
buckets_to_list(response_file['aggregations'].items(),'FILE',data1)
buckets_to_list(response_reg['aggregations'].items(),'REG',data1)
buckets_to_list(response_log['aggregations'].items(),'LOG',data1)

data1

[['FILE', 'NTFS USN change', 292405],
 ['FILE', 'File stat', 1880],
 ['FILE', 'File entry shell item', 25],
 ['REG', 'Registry Key', 18055],
 ['REG', 'Task Cache', 158],
 ['REG', 'Registry Key - Service', 65],
 ['REG', 'Background Activity Moderator Registry Entry', 36],
 ['REG', 'AppCompatCache Registry Entry', 13],
 ['REG', 'Registry Key - Winlogon', 3],
 ['REG', 'Registry Key - USB Entries', 2],
 ['REG', 'Registry Key - Run Key', 1],
 ['REG', 'Registry Key - User Account Information', 1],
 ['REG', 'Registry Key Shutdown Entry', 1],
 ['LOG', 'WinPrefetch', 597],
 ['LOG', 'Windows Setupapi Log', 18],
 ['LOG', 'System', 17],
 ['LOG', 'System - Network Connection', 1]]

In [ ]:
df1 = pd.DataFrame(data1)
df1

,0,1,2
0,FILE,NTFS USN change,292405
1,FILE,File stat,1880
2,FILE,File entry shell item,25
3,REG,Registry Key,18055
4,REG,Task Cache,158
5,REG,Registry Key - Service,65
6,REG,Background Activity Moderator Registry Entry,36
7,REG,AppCompatCache Registry Entry,13
8,REG,Registry Key - Winlogon,3
9,REG,Registry Key - USB Entries,2


In [ ]:
# set column names

df1.columns = ['source_short','source_long','count']
df1

,source_short,source_long,count
0,FILE,NTFS USN change,292405
1,FILE,File stat,1880
2,FILE,File entry shell item,25
3,REG,Registry Key,18055
4,REG,Task Cache,158
5,REG,Registry Key - Service,65
6,REG,Background Activity Moderator Registry Entry,36
7,REG,AppCompatCache Registry Entry,13
8,REG,Registry Key - Winlogon,3
9,REG,Registry Key - USB Entries,2


In [ ]:
!pip install --upgrade plotly

import plotly.express as px

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 5.2 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [ ]:
fig = px.treemap(df1, path=['source_short', 'source_long'],values='count')
fig.show()

This was a bit cumbersome approach. In step 3 we'll improve and show a way how to do this with a single opensearch query

####Step 3
Opensearch can also aggregate accros multiple fields combined. This is called multi_terms aggregation which aggregates accross all (source_long, source_short) value pairs. Note: by default opensearch returns 10 buckets max. We set it to 20 here using the size parameter.

See https://opensearch.org/docs/2.0/opensearch/bucket-agg/ for more information about opensearch and multi_term aggregation.

In [ ]:
facets = '"aggs": {   "source_long_short": { "multi_terms": { "terms": [{ "field": "source_long.keyword"},{"field": "source_short.keyword"}], "size":20}}}'
daterange = 'datetime:[2019-03-12 TO 2019-03-22]'
# FILE:
querystring = '{ "query_string": {"query": "%s AND  source_short:(FILE OR LOG OR REG)"  }}' % daterange
query = '{"query": %s,%s}' % (querystring,facets)

response = client.search(index="newmus2019ctf", body=query, size=0,track_total_hits=True)
response

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 313278, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'source_long_short': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': ['NTFS USN change', 'FILE'],
     'key_as_string': 'NTFS USN change|FILE',
     'doc_count': 292405},
    {'key': ['Registry Key', 'REG'],
     'key_as_string': 'Registry Key|REG',
     'doc_count': 18055},
    {'key': ['File stat', 'FILE'],
     'key_as_string': 'File stat|FILE',
     'doc_count': 1880},
    {'key': ['WinPrefetch', 'LOG'],
     'key_as_string': 'WinPrefetch|LOG',
     'doc_count': 597},
    {'key': ['Task Cache', 'REG'],
     'key_as_string': 'Task Cache|REG',
     'doc_count': 158},
    {'key': ['Registry Key - Service', 'REG'],
     'key_as_string': 'Registry Key - Service|REG',
     'doc_count': 65},
    {'key': ['Background Activity Moderator Regist

In [ ]:
# we need to redefine our bucket to list function

def buckets_to_list2(agg_dict):
  list = []
  for field, val in agg_dict:
    for bucket in val['buckets']:
      list.append([bucket['key'][1],bucket['key'][0],bucket['doc_count']])
  return list


data2 = buckets_to_list2(response['aggregations'].items())

In [ ]:
data2


[['FILE', 'NTFS USN change', 292405],
 ['REG', 'Registry Key', 18055],
 ['FILE', 'File stat', 1880],
 ['LOG', 'WinPrefetch', 597],
 ['REG', 'Task Cache', 158],
 ['REG', 'Registry Key - Service', 65],
 ['REG', 'Background Activity Moderator Registry Entry', 36],
 ['FILE', 'File entry shell item', 25],
 ['LOG', 'Windows Setupapi Log', 18],
 ['LOG', 'System', 17],
 ['REG', 'AppCompatCache Registry Entry', 13],
 ['REG', 'Registry Key - Winlogon', 3],
 ['REG', 'Registry Key - USB Entries', 2],
 ['REG', 'Registry Key - Run Key', 1],
 ['REG', 'Registry Key - User Account Information', 1],
 ['REG', 'Registry Key Shutdown Entry', 1],
 ['LOG', 'System - Network Connection', 1]]

In [ ]:
df2 = pd.DataFrame(data2)
df2.columns = ['source_short','source_long','count']
df2

,source_short,source_long,count
0,FILE,NTFS USN change,292405
1,REG,Registry Key,18055
2,FILE,File stat,1880
3,LOG,WinPrefetch,597
4,REG,Task Cache,158
5,REG,Registry Key - Service,65
6,REG,Background Activity Moderator Registry Entry,36
7,FILE,File entry shell item,25
8,LOG,Windows Setupapi Log,18
9,LOG,System,17


In [ ]:
fig = px.treemap(df2, path=['source_short', 'source_long'],values='count')
fig.show()

So we have the same result as under step 2 but we only needed one query by using multi_term aggregation.

####Step 4
The exercise asked for aggregation accross 3 fields. So let's add the parser field and deepen our treemap visualisation

In [ ]:
facets = '"aggs": {   "source_long_short": { "multi_terms": { "terms": [{ "field": "source_long.keyword"},{"field": "source_short.keyword"},{ "field": "parser.keyword"}], "size":200}}}'
daterange = 'datetime:[2019-03-12 TO 2019-03-22]'
# FILE:
querystring = '{ "query_string": {"query": "%s "  }}' % daterange
query = '{"query": %s,%s}' % (querystring,facets)

response3 = client.search(index="newmus2019ctf", body=query, size=0,track_total_hits=True)
response3

{'took': 9,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 367946, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'source_long_short': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': ['NTFS USN change', 'FILE', 'usnjrnl'],
     'key_as_string': 'NTFS USN change|FILE|usnjrnl',
     'doc_count': 292405},
    {'key': ['WinEVTX', 'EVT', 'winevtx'],
     'key_as_string': 'WinEVTX|EVT|winevtx',
     'doc_count': 54118},
    {'key': ['Registry Key', 'REG', 'winreg/winreg_default'],
     'key_as_string': 'Registry Key|REG|winreg/winreg_default',
     'doc_count': 17919},
    {'key': ['File stat', 'FILE', 'filestat'],
     'key_as_string': 'File stat|FILE|filestat',
     'doc_count': 1880},
    {'key': ['WinPrefetch', 'LOG', 'prefetch'],
     'key_as_string': 'WinPrefetch|LOG|prefetch',
     'doc_count': 597},
    {'key': ['MSIE WebCache container record',
  

In [ ]:
def buckets_to_list3(agg_dict):
  list = []
  for field, val in agg_dict:
    for bucket in val['buckets']:
      list.append([bucket['key'][2],bucket['key'][1],bucket['key'][0],bucket['doc_count']])
  return list


data3 = buckets_to_list3(response3['aggregations'].items())
data3

[['usnjrnl', 'FILE', 'NTFS USN change', 292405],
 ['winevtx', 'EVT', 'WinEVTX', 54118],
 ['winreg/winreg_default', 'REG', 'Registry Key', 17919],
 ['filestat', 'FILE', 'File stat', 1880],
 ['prefetch', 'LOG', 'WinPrefetch', 597],
 ['esedb/msie_webcache', 'WEBHIST', 'MSIE WebCache container record', 435],
 ['winreg/windows_task_cache', 'REG', 'Task Cache', 158],
 ['winreg/amcache', 'REG', 'Registry Key', 131],
 ['winreg/windows_services', 'REG', 'Registry Key - Service', 65],
 ['esedb/msie_webcache', 'WEBHIST', 'MSIE WebCache containers record', 37],
 ['winreg/bam', 'REG', 'Background Activity Moderator Registry Entry', 36],
 ['olecf/olecf_automatic_destinations/lnk', 'LNK', 'Windows Shortcut', 23],
 ['esedb/msie_webcache', 'WEBHIST', 'MSIE WebCache partitions record', 18],
 ['setupapi', 'LOG', 'Windows Setupapi Log', 18],
 ['olecf/olecf_automatic_destinations/lnk/shell_items',
  'FILE',
  'File entry shell item',
  17],
 ['olecf/olecf_automatic_destinations', 'OLECF', 'OLECF Dest list 

In [ ]:
df3 = pd.DataFrame(data3)
df3.columns = ['parser','source_short','source_long','count']
df3

,parser,source_short,source_long,count
0,usnjrnl,FILE,NTFS USN change,292405
1,winevtx,EVT,WinEVTX,54118
2,winreg/winreg_default,REG,Registry Key,17919
3,filestat,FILE,File stat,1880
4,prefetch,LOG,WinPrefetch,597
5,esedb/msie_webcache,WEBHIST,MSIE WebCache container record,435
6,winreg/windows_task_cache,REG,Task Cache,158
7,winreg/amcache,REG,Registry Key,131
8,winreg/windows_services,REG,Registry Key - Service,65
9,esedb/msie_webcache,WEBHIST,MSIE WebCache containers record,37


In [ ]:
fig = px.treemap(df3, path=['parser','source_short', 'source_long'],values='count')
fig.show()